<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-LLM-ZeroToAll/06_LA_CAG.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 07. LA CAG (Credibility-Aware Generation)

## Overview  
In this exercise, we will explore Language Analysis (LA) combined with Credibility-Aware Generation (CAG) using the Solar framework. This notebook will demonstrate how to analyze language data for credibility and generate reliable outputs. The techniques covered will enhance the accuracy and trustworthiness of text generated from various language inputs.
 
## Purpose of the Exercise
The purpose of this exercise is to integrate Language Analysis with Credibility-Aware Generation to produce credible and well-analyzed outputs. By the end of this tutorial, users will be able to analyze text for credibility and apply these insights to generate reliable and accurate responses using the Solar framework.


# No.1 accuracy in multiform table extraction 
- Convert documents to maximize RAG performance 
- LangChain provides powerful tools for text splitting and vectorization


![Layout Analyzer](./figures/la.png)

In [1]:
! pip3 install -qU  markdownify  langchain-upstage==0.1.8rc0  requests  python-dotenv


[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
#@title set API key
from pprint import pprint
import os

import warnings
warnings.filterwarnings('ignore')

from IPython import get_ipython

upstage_api_key_env_name = 'UPSTAGE_API_KEY'
def load_env():
    if 'google.colab' in str(get_ipython()):
        # Running in Google Colab
        from google.colab import userdata
        upstage_api_key = userdata.get(upstage_api_key_env_name)
        return os.environ.setdefault('UPSTAGE_API_KEY', upstage_api_key)
    else:
        # Running in local Jupyter Notebook
        from dotenv import load_dotenv
        load_dotenv()
        return os.environ.get(upstage_api_key_env_name)

UPSTAGE_API_KEY = load_env()

Enter your API Key ········


![Layout Analyzer](./figures/solar_sample.png)

In [3]:
from langchain_upstage import UpstageLayoutAnalysisLoader


layzer = UpstageLayoutAnalysisLoader("pdfs/solar_sample.pdf", output_type="html")
# For improved memory efficiency, consider using the lazy_load method to load documents page by page.
docs = layzer.load()

In [17]:
docs = layzer.alazy_load()

TypeError: async_generator.asend() takes exactly one argument (0 given)

In [15]:
from IPython.display import display, HTML

display(HTML(docs[0].page_content[:5000]))

TypeError: 'async_generator' object is not subscriptable

In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage


llm = ChatUpstage()

prompt_template = PromptTemplate.from_template(
    """
    Please provide most correct answer from the following context. 
    Think step by step and look the html tags and table values carefully to provide the most correct answer.
    If the answer is not present in the context, please write "The information is not present in the context."
    ---
    Question: {question}
    ---
    Context: {Context}
    """
)
chain = prompt_template | llm | StrOutputParser()

In [7]:
chain.invoke({"question": "Explain Table 2?", "Context": docs})

'Table 2 presents the evaluation results for the Open LLM Leaderboard, which includes SOLAR 10.7B and SOLAR 10.7B-Instruct along with other top-performing models. The table reports the scores for six tasks mentioned in Sec. 4.1, along with the H6 score (average of six tasks). The table also provides information on the size of the models in billions of parameters and the type of training stage, which can be chosen from {Pretrained, Instruction-tuned, Alignment-tuned}. Models based on SOLAR 10.7B are colored purple, and the best scores for H6 and the individual tasks are shown in bold.'

In [8]:
chain.invoke({"question": "What is MMLU scores of SOLAR 10.7B?", "Context": docs})

'The MMLU scores of SOLAR 10.7B is 65.48.'

In [9]:
chain.invoke({"question": "What is MMLU scores of Mistral 7B-Instruct-v0.2?", "Context": docs})

'MMLU scores of Mistral 7B-Instruct-v0.2 is 60.78.'

# Excercise 
Sometimes, even if we provide a table in Markdown or HTML format, the Large Language Model (LLM) may not extract the information correctly. How can you fix this issue?

Hint: Consider using CoT, a few-shot learning approach or a divide and conquer strategy. 
